In [2]:
import duckdb


In [3]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [32]:
df = con.execute("SELECT * FROM bronze_z0019").fetchdf()
df.head(10)

,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao
0,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-03-17 21:41:18.368795
1,10002,MARTELO,BT50,100,1500,z0019_1.csv,2025-03-17 21:41:18.368795
2,10003,PREGO,BT10,100,50,z0019_1.csv,2025-03-17 21:41:18.368795
3,10004,SERRA,BT50,100,200,z0019_2.csv,2025-03-17 21:50:12.561581
4,10005,MACHADO,BT50,100,100,z0019_2.csv,2025-03-17 21:50:12.561581
5,10003,PREGO,BT10,100,60,z0019_2.csv,2025-03-17 21:50:12.561581


In [33]:
df = con.execute("""
    SELECT * FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row 
    FROM bronze_z0019
    WHERE 1 = 1
    AND data_ingestao >= '2025-03-17'
                 ) WHERE row = 1

""").fetchdf()

df.head(10)


,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,row
0,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-03-17 21:41:18.368795,1
1,10004,SERRA,BT50,100,200,z0019_2.csv,2025-03-17 21:50:12.561581,1
2,10002,MARTELO,BT50,100,1500,z0019_1.csv,2025-03-17 21:41:18.368795,1
3,10005,MACHADO,BT50,100,100,z0019_2.csv,2025-03-17 21:50:12.561581,1
4,10003,PREGO,BT10,100,60,z0019_2.csv,2025-03-17 21:50:12.561581,1


In [35]:
df.head(10)

,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,row
0,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-03-17 21:41:18.368795,1
1,10004,SERRA,BT50,100,200,z0019_2.csv,2025-03-17 21:50:12.561581,1
2,10002,MARTELO,BT50,100,1500,z0019_1.csv,2025-03-17 21:41:18.368795,1
3,10005,MACHADO,BT50,100,100,z0019_2.csv,2025-03-17 21:50:12.561581,1
4,10003,PREGO,BT10,100,60,z0019_2.csv,2025-03-17 21:50:12.561581,1


In [38]:
df_final = df.drop(columns=['nome_arquivo','data_ingestao', 'row'])
df_final = df_final.rename(columns={"NATBR":"id"})
df_final = df_final.rename(columns={"MAKTX":"nm_produto"})
df_final = df_final.rename(columns={"WERKS":"id_categoria"})
df_final = df_final.rename(columns={"MAINS":"id_fornecedor"})
df_final = df_final.rename(columns={"LABST":"vl_preco"})
df_final.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100
1,10004,SERRA,BT50,100,200
2,10002,MARTELO,BT50,100,1500
3,10005,MACHADO,BT50,100,100
4,10003,PREGO,BT10,100,60


In [40]:
df_final.dtypes

id               object
nm_produto       object
id_categoria     object
id_fornecedor    object
vl_preco         object
dtype: object

In [50]:
df2 = df_final 
df2 = df2.astype(
    {
        "id": int,
        "nm_produto": str,
        "id_categoria": str,
        "id_fornecedor": int,
        "vl_preco": float
    }

)
df2.dtypes
#df2.head(10)

id                 int64
nm_produto        object
id_categoria      object
id_fornecedor      int64
vl_preco         float64
dtype: object

In [52]:
con.execute("""
CREATE TABLE IF NOT EXISTS produtos (
            id BIGINT,
            nm_produto TEXT,
            id_categoria TEXT,
            id_fornecedor BIGINT,
            vl_preco FLOAT
            )
""")

In [53]:
df2.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100.0
1,10004,SERRA,BT50,100,200.0
2,10002,MARTELO,BT50,100,1500.0
3,10005,MACHADO,BT50,100,100.0
4,10003,PREGO,BT10,100,60.0


In [55]:
con.execute("INSERT INTO produtos SELECT * FROM df2")

In [56]:
df_resultado = con.execute("SELECT * FROM produtos").fetchdf()
df_resultado.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100.0
1,10004,SERRA,BT50,100,200.0
2,10002,MARTELO,BT50,100,1500.0
3,10005,MACHADO,BT50,100,100.0
4,10003,PREGO,BT10,100,60.0


In [57]:
con.close()